In [1]:
import pandas

In [14]:
import torch
import torch.nn as nn
import torch.optim as optim

from torchvision import datasets , transforms

from torch.utils.data import DataLoader

In [6]:
torch.cuda.is_available()   # gpu is not available

False

In [8]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

In [10]:
transform = transforms.Compose([
    transforms.Resize((150,150)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize([0.5]*3,[0.5]*3)
])

In [11]:
train_dataset = datasets.ImageFolder("data/train",transform=transform)

In [12]:
train_dataset

Dataset ImageFolder
    Number of datapoints: 836
    Root location: data/train
    StandardTransform
Transform: Compose(
               Resize(size=(150, 150), interpolation=bilinear, max_size=None, antialias=True)
               RandomHorizontalFlip(p=0.5)
               RandomRotation(degrees=[-10.0, 10.0], interpolation=nearest, expand=False, fill=0)
               ToTensor()
               Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
           )

In [13]:
val_dataset = datasets.ImageFolder("data/val",transform=transform)

In [15]:
train_loader = DataLoader(train_dataset,batch_size=16,shuffle=True)
val_loader = DataLoader(val_dataset,batch_size=16)

In [33]:
con_layers = nn.Sequential(
    
        nn.Conv2d(3,32,3), nn.ReLU(), nn.MaxPool2d(2),
        nn.Conv2d(32,64,3), nn.ReLU(), nn.MaxPool2d(2),
        nn.Conv2d(64,128,3), nn.ReLU(), nn.MaxPool2d(2)
    
)

In [17]:
with torch.no_grad():
    dummy_input = torch.randn(1,3,150,150)
    dummy_output = con_layers(dummy_input)
    flatten_dim = dummy_output.view(1,-1).shape[1]

In [34]:
model = nn.Sequential(
    con_layers,
    nn.Flatten(),
    nn.Linear(flatten_dim,256),nn.ReLU(),
    nn.Dropout(0.3),
    nn.Linear(256,1),
    nn.Sigmoid()
).to(device)

In [35]:
def init_weights(m):
    if isinstance(m,nn.Conv2d) or isinstance(m,nn.Linear):
        nn.init.kaiming_normal_(m.weight)

In [36]:
model.apply(init_weights)

Sequential(
  (0): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
    (4): ReLU()
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1))
    (7): ReLU()
    (8): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (1): Flatten(start_dim=1, end_dim=-1)
  (2): Linear(in_features=36992, out_features=256, bias=True)
  (3): ReLU()
  (4): Dropout(p=0.3, inplace=False)
  (5): Linear(in_features=256, out_features=1, bias=True)
  (6): Sigmoid()
)

In [37]:
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(),lr = 0.001)

In [38]:
for epochs in range(2):
    model.train()
    total_loss = 0
    for inputs ,labels in train_loader:
        inputs,labels = inputs.to(device),labels.float().to(device).view(-1,1)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs,labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
        print(epochs +1 , total_loss)

1 0.9528599977493286
1 31.52790081501007
1 67.03620541095734
1 104.53620541095734
1 173.28620541095734
1 210.78620541095734
1 260.78620541095734
1 304.53620541095734
1 354.53620541095734
1 423.28620541095734
1 454.53620541095734
1 498.28620541095734
1 517.0362054109573
1 560.7862054109573
1 592.0362054109573
1 629.5362054109573
1 679.5362054109573
1 735.7862054109573
1 754.5362054109573
1 792.0362054109573
1 829.5362054109573
1 867.0362054109573
1 898.2862054109573
1 942.0362054109573
1 992.0362054109573
1 1054.5362054109573
1 1085.7862054109573
1 1160.7862054109573
1 1192.0362054109573
1 1254.5362054109573
1 1292.0362054109573
1 1342.0362054109573
1 1367.0362054109573
1 1423.2862054109573
1 1460.7862054109573
1 1510.7862054109573
1 1548.2862054109573
1 1598.2862054109573
1 1629.5362054109573
1 1679.5362054109573
1 1704.5362054109573
1 1742.0362054109573
1 1785.7862054109573
1 1842.0362054109573
1 1910.7862054109573
1 1985.7862054109573
1 2029.5362054109573
1 2079.5362054109573
1 2117.

In [39]:
torch.save(model.state_dict(),"name.pth")